In [4]:
from IPython import get_ipython
from IPython.display import display
import asyncio
# %%
# @title
# Note: This is conceptual code and requires installation of libraries and adjustments

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import requests
import os
import asyncio
import time
import requests
from requests.exceptions import RequestException
import nest_asyncio  # Import nest_asyncio

# Check if playwright is installed and install if not
try:
    import playwright
except ImportError:
    print("Playwright is not installed. Installing...")
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'playwright'], check=True)
        subprocess.run([sys.executable, '-m', 'playwright', 'install'], check=True)
        import playwright
    except subprocess.CalledProcessError as e:
        print(f"Error installing playwright: {e}")
        print("Please make sure you have pip installed and try installing playwright manually: pip install playwright && playwright install")
        sys.exit(1)
    except Exception as e:
        print(f"An unexpected error occurred during playwright installation: {e}")
        sys.exit(1)
else:
    print("Playwright is already installed.")

def get_links_from_page(url, max_retries=3, retry_delay=5):
    """Extracts all links from a given webpage URL within the same domain."""
    retries = 0
    retries < max_retries
    try:
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.content, 'html.parser')
        links = set()
        base_url = url.split('/')[0] + '//' + url.split('/')[2] # Extract base URL

        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if href.startswith('/') or href.startswith(base_url): # Check if link is within the same domain
                if href.startswith('/'):
                    absolute_url = base_url + href
                else:
                    absolute_url = href
                links.add(absolute_url)
        return links
    except RequestException as e:
            if response.status_code == 522 and retries < max_retries:
                print(f"Error fetching URL {url}: {e}, Retry {retries+1}/{max_retries} in {retry_delay} seconds...")
                retries += 1
                time.sleep(retry_delay)
            else:
                print(f"Error fetching URL {url}: {e}")
                return set()  # Return empty set if not a 522 or max retries reached

    print(f"Max retries reached for URL {url}. Skipping...")
    return set() # Return empty set after max retries

def crawl_website(start_url):
    """Crawls a website and returns a set of unique URLs."""
    crawled_urls = set()
    urls_to_crawl = {start_url} # Use a set to avoid duplicates

    while urls_to_crawl:
        current_url = urls_to_crawl.pop()
        if current_url in crawled_urls:
            continue
        print(f"Crawling: {current_url}") # Optional: Print URLs as they are crawled
        crawled_urls.add(current_url)
        new_links = get_links_from_page(current_url)
        urls_to_crawl.update(new_links - crawled_urls) # Add only new links

    return crawled_urls


async def take_screenshot(url, device_type, output_folder="screenshots"):
    """Takes a screenshot of a URL in specified device emulation."""
    device_settings = {
        "mobile": {"viewport": {"width": 375, "height": 667}, "user_agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1"}, # Example iPhone SE
        "tablet": {"viewport": {"width": 768, "height": 1024}, "user_agent": "Mozilla/5.0 (iPad; CPU OS 11_0 like Mac OS X) AppleWebKit/604.1.34 (KHTML, like Gecko) Version/11.0 Mobile/15A5341f Safari/604.1"}, # Example iPad
        "desktop": {"viewport": {"width": 1440, "height": 900}, "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}  # Example Desktop
        }

    if device_type not in device_settings:
        raise ValueError("Invalid device type. Choose 'mobile' or 'tablet'.")

    config = device_settings[device_type]
    filename = f"{output_folder}/{url.replace('/', '_').replace(':', '_')}_{device_type}.png" # Create filename from URL

    os.makedirs(output_folder, exist_ok=True) # Ensure output folder exists

    async with async_playwright() as p: # Use async_playwright
        browser = await p.chromium.launch(headless=True) # Launch headless browser
        context = await browser.new_context(viewport=config["viewport"], user_agent=config["user_agent"]) # Set viewport and user agent
        page = await context.new_page()
        try:
            await page.goto(url, timeout=60000) # Increased timeout for page load
            await page.screenshot(path=filename, full_page=True) # Take full page screenshot
            print(f"Screenshot saved: {filename}")
        except Exception as e:
            print(f"Error taking screenshot for {url} ({device_type}): {e}")
        finally:
            await context.close()
            await browser.close()


async def main():
    start_website_url = "https://www.world-kinect.com/blog"
    output_dir = "blog"

    all_website_links = crawl_website(start_website_url)

    for link in all_website_links:
        await take_screenshot(link, "mobile", output_folder=output_dir)
        await take_screenshot(link, "tablet", output_folder=output_dir)
        await take_screenshot(link, "desktop", output_folder=output_dir)

    print(f"Screenshots saved to '{output_dir}' directory.")

if __name__ == "__main__":
    # Apply nest_asyncio to allow nested event loops
    nest_asyncio.apply()
    asyncio.run(main()) # Run the main function using asyncio
# %%

Playwright is already installed.
Crawling: https://www.world-kinect.com/blog
Crawling: https://www.world-kinect.com/energy-services/energy-supply
Crawling: https://www.world-kinect.com/blog/category/water
Crawling: https://www.world-kinect.com/about-us/qhsse
Crawling: https://www.world-kinect.com/taxonomy/term/1763
Crawling: https://www.world-kinect.com/newsroom
Crawling: https://www.world-kinect.com/blog/understanding-pjm-capacity-market-ensuring-reliable-electricity-supply
Crawling: https://www.world-kinect.com/blog/category/sustainability
Crawling: https://www.world-kinect.com/fuel-lubricants
Crawling: https://www.world-kinect.com/blog/unlocking-demand-response-potential
Crawling: https://www.world-kinect.com/blog/category/renewable-energy
Crawling: https://www.world-kinect.com/blog/how-us-energy-market-works-guide-small-and-medium-sized-businesses
Crawling: https://www.world-kinect.com/sites/default/files/2025-04/WK-Norwegian-Transparency-Act-Statement.pdf


Crawling: https://www.world-kinect.com/about-us
Crawling: https://www.world-kinect.com/sustainability/decarbonization-planning
Crawling: https://www.world-kinect.com/case-studies-resources/top-takeaways-get-kinected
Crawling: https://www.world-kinect.com/fuel-lubricants/fleet-fueling
Crawling: https://www.world-kinect.com/case-studies-resources/natural-gas-physical-supply-case-study
Crawling: https://www.world-kinect.com/careers/jobs-for-military-service-members-and-veterans/aus-military-careers
Crawling: https://www.world-kinect.com/blog/category/energy-management
Crawling: https://www.world-kinect.com/energy-services/energy-brokerage
Crawling: https://www.world-kinect.com/energy-services/energy-procurement-services
Crawling: https://www.world-kinect.com/your-privacy-center
Crawling: https://www.world-kinect.com/blog/crucial-role-risk-management-era-energy-cost-volatility
Crawling: https://www.world-kinect.com/blog/strategic-energy-relationship-john-innes-centre
Crawling: https://www.

<ipython-input-4-06011c60b128>:44: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.content, 'html.parser')


Crawling: https://www.world-kinect.com/blog/category/customer-story
Crawling: https://www.world-kinect.com/blog/category/compliance
Crawling: https://www.world-kinect.com/fuel-lubricants/fuel-retailer-solutions
Crawling: https://www.world-kinect.com/blog/8-key-definitions-help-you-understand-virtual-power-purchase-agreements-vppas
Crawling: https://www.world-kinect.com/case-studies-resources/cop27-top-10-takeaways
Crawling: https://www.world-kinect.com/blog/guide-switching-energy-suppliers-get-best-deal
Crawling: https://www.world-kinect.com/fuel-lubricants/home-heating-oil
Crawling: https://www.world-kinect.com/blog/cop26-developments
Crawling: https://www.world-kinect.com/reduce-carbon-footprint/offset-remaining-carbon-emissions
Crawling: https://www.world-kinect.com/energy-services/price-risk-management
Crawling: https://www.world-kinect.com/fuel-lubricants/lubricants-def-adblue
Crawling: https://www.world-kinect.com/blog/apprehender-la-gestion-denergie-en-periode-dinstabilite-geopo

Crawling: https://www.world-kinect.com/fuel-lubricants/bulk-fuel-delivery
Crawling: https://www.world-kinect.com/blog/tap-water-savings
Crawling: https://www.world-kinect.com/minimize-risk/price-risk-management
Crawling: https://www.world-kinect.com/sites/default/files/2024-04/WorldKinect_RetailFuel_Korean.pdf


...

Crawling: https://www.world-kinect.com/fuel-lubricants/fleetconnect
Crawling: https://www.world-kinect.com/fr/strategie-de-decarbonation/certificats-attribut-energetique
Crawling: https://www.world-kinect.com
Crawling: https://www.world-kinect.com/sites/default/files/2024-08/2022-Sustainability-Report-Highlights-compressed.pdf


Crawling: https://www.world-kinect.com/fr/blog/difference-entre-edf-et-enedis-tout-savoir-sur-leurs-missions
Crawling: https://www.world-kinect.com/cdn-cgi/l/email-protection#1e7d71706a7f7d6a5e69716c727a337577707b7d6a307d7173
Crawling: https://www.world-kinect.com/case-studies-resources/fleetconnect-tank-telemonitoring-case-study
Crawling: https://www.world-kinect.com/fr/blog/changer-de-fournisseur-energie-via-appel-offres
Crawling: https://www.world-kinect.com/fr/conseils-energie/conseils-en-achats-energie
Crawling: https://www.world-kinect.com/case-studies-resources/sourcing-renewable-energy-certificates-case-study-aviation
Crawling: https://www.world-kinect.com/cdn-cgi/l/email-protection#7d0d0e09180b18130e3d0a120f111950161413181e09531e12105d
Crawling: https://www.world-kinect.com/case-studies-resources/bulk-fuel-delivery-case-study-video
Crawling: https://www.world-kinect.com/case-studies-resources/carbon-compliance-services-esos-case-study
Crawling: https://www.world-kinect.com/sou

Crawling: https://www.world-kinect.com/fr/blog/category/marche-de-l-energie-et-acteurs
Crawling: https://www.world-kinect.com/fr/blog/choisir-le-bon-courtier-energie
Crawling: https://www.world-kinect.com/fr/blog/category/courtage-en-energie
Crawling: https://www.world-kinect.com/case-studies-resources/New-CA-Climate-Disclosure-Laws
Crawling: https://www.world-kinect.com/about-us/contact-us/portal-access-form
Crawling: https://www.world-kinect.com/cdn-cgi/l/email-protection#cc849e8fa3a1bca0a5ada2afa98cbbaabfafa3bebce2afa3a1
Crawling: https://www.world-kinect.com/fr/blog/comment-anticiper-le-renouvellement-de-votre-contrat-denergie-professionnel
Crawling: https://www.world-kinect.com/taxonomy/term/1827/feed
Crawling: https://www.world-kinect.com/sustainability/onsite-solar 
Error fetching URL https://www.world-kinect.com/sustainability/onsite-solar : 400 Client Error: Bad Request for url: https://www.world-kinect.com/sustainability/onsite-solar%20
Crawling: https://www.world-kinect.com/

Crawling: https://www.world-kinect.com//www.world-kinect.com/fr/contactez-nous
Error fetching URL https://www.world-kinect.com//www.world-kinect.com/fr/contactez-nous: 404 Client Error: Not Found for url: https://www.world-kinect.com//www.world-kinect.com/fr/contactez-nous
Crawling: https://www.world-kinect.com/fr/energie
Crawling: https://www.world-kinect.com/cdn-cgi/l/email-protection#4f2621392a3c3b203d0f38203d232b2426212a2c3b612c2022
Crawling: https://www.world-kinect.com/commercial-fleet-fuel-card-cfn-and-fuelman-case-study
Crawling: https://www.world-kinect.com/fr/gestion-des-risques-lies-aux-prix-de-lenergie
Crawling: https://www.world-kinect.com/taxonomy/term/1739
Crawling: https://www.world-kinect.com/ventures/approach
Crawling: https://www.world-kinect.com/taxonomy/term/1738/feed
Crawling: https://www.world-kinect.com/fr/taxonomy/term/1794/feed
Crawling: https://www.world-kinect.com/taxonomy/term/1733
Crawling: https://www.world-kinect.com/minimize-risk/data-visibility
Crawlin

Crawling: https://www.world-kinect.com/producers-trust
Crawling: https://www.world-kinect.com/taxonomy/term/1737
Crawling: https://www.world-kinect.com/taxonomy/term/1739/feed
Crawling: https://www.world-kinect.com/energy-impact-partners
Crawling: https://www.world-kinect.com/blog/california-climate-laws
Crawling: https://www.world-kinect.com/media/5041
Crawling: https://www.world-kinect.com/taxonomy/term/1737/feed
Crawling: https://www.world-kinect.com/Ventures/contact-our-ventures-team
Crawling: https://www.world-kinect.com/taxonomy/term/1734
Crawling: https://www.world-kinect.com/sparkcharge
Crawling: https://www.world-kinect.com/meld-energy
Crawling: https://www.world-kinect.com/taxonomy/term/1735/feed
Crawling: https://www.world-kinect.com/taxonomy/term/1736
Crawling: https://www.world-kinect.com/taxonomy/term/1733/feed
Crawling: https://www.world-kinect.com/Ventures
Crawling: https://www.world-kinect.com/taxonomy/term/1734/feed
Crawling: https://www.world-kinect.com/taxonomy/term

In [3]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 26.2 MB/s eta 0:00:00
167.7 MiB [] 0% 0.0s167.7 MiB [] 0% 8.4s167.7 MiB [] 0% 4.9s167.7 MiB [] 1% 3.7s167.7 MiB [] 1% 3.4s167.7 MiB [] 2% 3.4s167.7 MiB [] 3% 2.8s167.7 MiB [] 4% 2.5s167.7 MiB [] 5% 2.3s167.7 MiB [] 5% 2.6s167.7 MiB [] 6% 2.5s167.7 MiB [] 7% 2.4s167.7 MiB [] 7% 2.3s167.7 MiB [] 8% 2.2s167.7 MiB [] 9% 2.2s167.7 MiB [] 10% 2.0s167.7 MiB [] 10% 2.1s167.7 MiB [] 12% 2.0s167.7 MiB [] 13% 1.9s167.7 MiB [] 14% 1.9s167.7 MiB [] 15% 1.8s167.7 MiB [] 16% 1.7s167.7 MiB [] 17% 1.7s167.7 MiB [] 18% 1.7s167.7 MiB [] 19% 1.6s167.7 MiB [] 20% 1.6s167.7 MiB [] 21% 1.5s167.7 MiB [] 21% 1.6s167.7 MiB [] 22% 1.6s167.7 MiB [] 23% 1.6s167.7 MiB [] 24% 1.5s167.7 MiB [] 25% 1.5s167.7 MiB [] 27% 1.4s167.7 MiB [] 28% 1.4s167.7 MiB [] 29% 1.3s167.7 MiB [] 30% 1.3s167.7 MiB [] 31% 1.3s167.7 MiB [] 32% 1.3s167.7 MiB [] 33% 1.3s167.7 MiB [] 34% 1.2s167.7 MiB [] 36% 1.2s167.7 MiB [] 37% 1.2s167.7 MiB [] 38% 1.1s167.7 MiB [] 39% 1.1s167.7 MiB []

# New Section

In [ ]:
!playwright install

/bin/bash: line 1: playwright: command not found


In [5]:
!cp -r /content/blog /content/drive/MyDrive/
